# Exploratory Data Analysis: Olist E-Commerce Dataset

**Objective**: Understand the data structure, quality, and behavioral patterns to inform customer clustering for persona-based behavioral simulation.

## Dataset Overview

The Olist Brazilian E-Commerce dataset contains ~100K orders from 2016-2018 across 9 relational tables:
- Customer demographics and identifiers
- Order transactions with timestamps and status
- Line-item details (products, prices, freight)
- Payment information (method, installments)
- Customer reviews and ratings
- Product catalog with categories
- Seller information
- Geographic coordinates by ZIP code

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# Style settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

DATA_DIR = Path('../data/raw')

## 1. Data Loading

In [ ]:
# Load all datasets
customers = pd.read_csv(DATA_DIR / 'olist_customers_dataset.csv')

orders = pd.read_csv(
    DATA_DIR / 'olist_orders_dataset.csv',
    parse_dates=['order_purchase_timestamp', 'order_approved_at',
                 'order_delivered_carrier_date', 'order_delivered_customer_date',
                 'order_estimated_delivery_date']
)

order_items = pd.read_csv(
    DATA_DIR / 'olist_order_items_dataset.csv',
    parse_dates=['shipping_limit_date']
)

payments = pd.read_csv(DATA_DIR / 'olist_order_payments_dataset.csv')

reviews = pd.read_csv(
    DATA_DIR / 'olist_order_reviews_dataset.csv',
    parse_dates=['review_creation_date', 'review_answer_timestamp']
)

products = pd.read_csv(DATA_DIR / 'olist_products_dataset.csv')

sellers = pd.read_csv(DATA_DIR / 'olist_sellers_dataset.csv')

geolocation = pd.read_csv(DATA_DIR / 'olist_geolocation_dataset.csv')

category_translation = pd.read_csv(DATA_DIR / 'product_category_name_translation.csv')

print('All datasets loaded successfully.')

## 2. Dataset Profiling

### 2.1 Profiling Function

In [ ]:
def profile_dataset(df, name):
    """Generate comprehensive profile for a dataframe."""
    print(f"\n{'='*60}")
    print(f"DATASET: {name}")
    print(f"{'='*60}")
    print(f"Rows: {len(df):,}")
    print(f"Columns: {len(df.columns)}")
    print(f"Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"Duplicate rows: {df.duplicated().sum():,}")
    print(f"\nColumn Details:")
    print("-" * 60)
    
    col_stats = []
    for col in df.columns:
        non_null = df[col].notna().sum()
        null_count = df[col].isna().sum()
        null_pct = df[col].isna().mean() * 100
        
        # Check for empty strings in object columns
        empty_str = 0
        if df[col].dtype == 'object':
            empty_str = (df[col] == '').sum()
        
        col_stats.append({
            'column': col,
            'dtype': str(df[col].dtype),
            'non_null': non_null,
            'null_count': null_count,
            'null_pct': null_pct,
            'empty_str': empty_str,
            'unique': df[col].nunique()
        })
    
    stats_df = pd.DataFrame(col_stats)
    print(stats_df.to_string(index=False))
    return stats_df

# Store all profiles
profiles = {}

### 2.2 Customers Dataset

In [ ]:
profiles['customers'] = profile_dataset(customers, 'olist_customers_dataset.csv')
print("\nSample rows:")
display(customers.head(3))

### 2.3 Orders Dataset

In [ ]:
profiles['orders'] = profile_dataset(orders, 'olist_orders_dataset.csv')
print("\nSample rows:")
display(orders.head(3))
print("\nOrder status distribution:")
print(orders['order_status'].value_counts())

### 2.4 Order Items Dataset

In [ ]:
profiles['order_items'] = profile_dataset(order_items, 'olist_order_items_dataset.csv')
print("\nSample rows:")
display(order_items.head(3))
print("\nPrice statistics:")
print(order_items[['price', 'freight_value']].describe())

### 2.5 Payments Dataset

In [ ]:
profiles['payments'] = profile_dataset(payments, 'olist_order_payments_dataset.csv')
print("\nSample rows:")
display(payments.head(3))
print("\nPayment type distribution:")
print(payments['payment_type'].value_counts())
print("\nInstallments distribution:")
print(payments['payment_installments'].value_counts().head(10))

### 2.6 Reviews Dataset

In [ ]:
profiles['reviews'] = profile_dataset(reviews, 'olist_order_reviews_dataset.csv')
print("\nSample rows:")
display(reviews.head(3))
print("\nReview score distribution:")
print(reviews['review_score'].value_counts().sort_index())

### 2.7 Products Dataset

In [ ]:
profiles['products'] = profile_dataset(products, 'olist_products_dataset.csv')
print("\nSample rows:")
display(products.head(3))

# Merge with translations
products = products.merge(category_translation, on='product_category_name', how='left')
print("\nTop 10 product categories (English):")
print(products['product_category_name_english'].value_counts().head(10))

### 2.8 Sellers Dataset

In [ ]:
profiles['sellers'] = profile_dataset(sellers, 'olist_sellers_dataset.csv')
print("\nSample rows:")
display(sellers.head(3))
print("\nTop seller states:")
print(sellers['seller_state'].value_counts().head(10))

### 2.9 Geolocation Dataset

In [ ]:
profiles['geolocation'] = profile_dataset(geolocation, 'olist_geolocation_dataset.csv')
print("\nSample rows:")
display(geolocation.head(3))
print(f"\nUnique ZIP prefixes: {geolocation['geolocation_zip_code_prefix'].nunique():,}")

## 3. Missing Data Analysis

In [ ]:
def comprehensive_missing_analysis(df, name):
    """Detect all forms of missing values."""
    null_patterns = ['', ' ', 'NULL', 'null', 'None', 'N/A', 'n/a', 'NA', '-']
    
    results = []
    for col in df.columns:
        pandas_null = df[col].isna().sum()
        empty_str = 0
        whitespace = 0
        
        if df[col].dtype == 'object':
            empty_str = (df[col] == '').sum()
            whitespace = df[col].astype(str).str.strip().eq('').sum() - empty_str - pandas_null
            whitespace = max(0, whitespace)
        
        total = pandas_null + empty_str + whitespace
        results.append({
            'column': col,
            'pandas_null': pandas_null,
            'empty_string': empty_str,
            'whitespace': whitespace,
            'total_missing': total,
            'pct_missing': total / len(df) * 100
        })
    
    return pd.DataFrame(results)

# Analyze key datasets
print("Missing Data Analysis")
print("="*60)

for name, df in [('orders', orders), ('reviews', reviews), ('products', products)]:
    missing = comprehensive_missing_analysis(df, name)
    missing_cols = missing[missing['total_missing'] > 0]
    if len(missing_cols) > 0:
        print(f"\n{name.upper()} - Columns with missing data:")
        print(missing_cols.to_string(index=False))

## 4. Outlier Detection

In [ ]:
def detect_outliers_iqr(df, columns, threshold=1.5):
    """Detect outliers using IQR method."""
    results = []
    for col in columns:
        if col not in df.columns:
            continue
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - threshold * IQR
        upper = Q3 + threshold * IQR
        outliers = df[(df[col] < lower) | (df[col] > upper)]
        results.append({
            'column': col,
            'min': df[col].min(),
            'Q1': Q1,
            'median': df[col].median(),
            'Q3': Q3,
            'max': df[col].max(),
            'IQR': IQR,
            'lower_bound': lower,
            'upper_bound': upper,
            'outlier_count': len(outliers),
            'outlier_pct': len(outliers) / len(df) * 100
        })
    return pd.DataFrame(results)

# Check order_items
print("Order Items - Outlier Analysis")
print("="*60)
outliers_items = detect_outliers_iqr(order_items, ['price', 'freight_value'])
print(outliers_items.to_string(index=False))

# Check payments
print("\n\nPayments - Outlier Analysis")
print("="*60)
outliers_payments = detect_outliers_iqr(payments, ['payment_value', 'payment_installments'])
print(outliers_payments.to_string(index=False))

## 5. Visualizations

### 5.1 Price and Freight Distributions

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Price distribution
axes[0, 0].hist(order_items['price'], bins=50, edgecolor='white', alpha=0.7)
axes[0, 0].set_xlabel('Price (BRL)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Price Distribution (Linear Scale)')

axes[0, 1].hist(order_items['price'], bins=50, edgecolor='white', alpha=0.7)
axes[0, 1].set_xscale('log')
axes[0, 1].set_xlabel('Price (BRL, log scale)')
axes[0, 1].set_title('Price Distribution (Log Scale)')

# Freight distribution
axes[1, 0].hist(order_items['freight_value'], bins=50, edgecolor='white', alpha=0.7, color='orange')
axes[1, 0].set_xlabel('Freight Value (BRL)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Freight Value Distribution')

# Price vs Freight scatter
sample = order_items.sample(min(5000, len(order_items)), random_state=42)
axes[1, 1].scatter(sample['price'], sample['freight_value'], alpha=0.3, s=10)
axes[1, 1].set_xlabel('Price (BRL)')
axes[1, 1].set_ylabel('Freight Value (BRL)')
axes[1, 1].set_title('Price vs Freight (sampled)')

plt.tight_layout()
plt.savefig('outputs/price_freight_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

### 5.2 Payment Type and Installments

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Payment type
payment_counts = payments['payment_type'].value_counts()
colors = sns.color_palette('husl', len(payment_counts))
axes[0].bar(payment_counts.index, payment_counts.values, color=colors)
axes[0].set_xlabel('Payment Type')
axes[0].set_ylabel('Count')
axes[0].set_title('Payment Type Distribution')
axes[0].tick_params(axis='x', rotation=45)

# Installments
installment_counts = payments['payment_installments'].value_counts().sort_index()
installment_counts = installment_counts[installment_counts.index <= 12]  # Focus on 1-12
axes[1].bar(installment_counts.index, installment_counts.values, color='teal')
axes[1].set_xlabel('Number of Installments')
axes[1].set_ylabel('Count')
axes[1].set_title('Installments Distribution (1-12)')

plt.tight_layout()
plt.savefig('outputs/payment_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

### 5.3 Review Score Distribution

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

review_counts = reviews['review_score'].value_counts().sort_index()
colors = ['#d73027', '#fc8d59', '#fee08b', '#d9ef8b', '#1a9850']  # Red to green
ax.bar(review_counts.index, review_counts.values, color=colors, edgecolor='white')
ax.set_xlabel('Review Score')
ax.set_ylabel('Count')
ax.set_title('Review Score Distribution')
ax.set_xticks([1, 2, 3, 4, 5])

# Add percentages
total = review_counts.sum()
for i, (score, count) in enumerate(review_counts.items()):
    ax.annotate(f'{count/total*100:.1f}%', (score, count), ha='center', va='bottom')

plt.tight_layout()
plt.savefig('outputs/review_score_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

### 5.4 Order Status and Timeline

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Order status
status_counts = orders['order_status'].value_counts()
axes[0].barh(status_counts.index, status_counts.values, color='steelblue')
axes[0].set_xlabel('Count')
axes[0].set_title('Order Status Distribution')

# Orders over time
orders_by_month = orders.set_index('order_purchase_timestamp').resample('M').size()
axes[1].plot(orders_by_month.index, orders_by_month.values, marker='o', linewidth=2)
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Number of Orders')
axes[1].set_title('Orders Over Time (Monthly)')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('outputs/order_status_timeline.png', dpi=150, bbox_inches='tight')
plt.show()

### 5.5 Product Categories

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

# Get top 20 categories
top_categories = products['product_category_name_english'].value_counts().head(20)
ax.barh(top_categories.index[::-1], top_categories.values[::-1], color='coral')
ax.set_xlabel('Number of Products')
ax.set_title('Top 20 Product Categories')

plt.tight_layout()
plt.savefig('outputs/top_categories.png', dpi=150, bbox_inches='tight')
plt.show()

### 5.6 Customer-Level Analysis

First, let's understand the relationship between `customer_id` and `customer_unique_id`.

In [ ]:
print("Customer ID Analysis")
print("="*60)
print(f"Total customer_id entries: {len(customers):,}")
print(f"Unique customer_id: {customers['customer_id'].nunique():,}")
print(f"Unique customer_unique_id: {customers['customer_unique_id'].nunique():,}")

# How many orders per unique customer?
orders_per_customer = customers.groupby('customer_unique_id').size()
print(f"\nOrders per unique customer:")
print(orders_per_customer.describe())
print(f"\nRepeat customers (>1 order): {(orders_per_customer > 1).sum():,} ({(orders_per_customer > 1).mean()*100:.2f}%)")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Orders per customer histogram
axes[0].hist(orders_per_customer.values, bins=range(1, 20), edgecolor='white', align='left')
axes[0].set_xlabel('Number of Orders')
axes[0].set_ylabel('Number of Customers')
axes[0].set_title('Purchase Frequency Distribution')
axes[0].set_xticks(range(1, 20))

# Customer state distribution
customer_states = customers.drop_duplicates('customer_unique_id')['customer_state'].value_counts().head(10)
axes[1].bar(customer_states.index, customer_states.values, color='purple')
axes[1].set_xlabel('State')
axes[1].set_ylabel('Number of Unique Customers')
axes[1].set_title('Top 10 Customer States')

plt.tight_layout()
plt.savefig('outputs/customer_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

### 5.7 Correlation Analysis

In [ ]:
# Merge key numeric features for correlation
correlation_df = order_items[['order_id', 'price', 'freight_value']].merge(
    payments[['order_id', 'payment_value', 'payment_installments']].groupby('order_id').agg({
        'payment_value': 'sum',
        'payment_installments': 'mean'
    }).reset_index(),
    on='order_id'
)

# Add review score
correlation_df = correlation_df.merge(
    reviews[['order_id', 'review_score']],
    on='order_id',
    how='left'
)

# Calculate correlation matrix
numeric_cols = ['price', 'freight_value', 'payment_value', 'payment_installments', 'review_score']
corr_matrix = correlation_df[numeric_cols].corr()

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0, fmt='.2f', 
            square=True, linewidths=0.5, ax=ax)
ax.set_title('Correlation Matrix of Order-Level Features')
plt.tight_layout()
plt.savefig('outputs/correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

### 5.8 Delivery Time Analysis

In [ ]:
# Calculate delivery metrics for delivered orders
delivered = orders[orders['order_status'] == 'delivered'].copy()
delivered['delivery_days'] = (delivered['order_delivered_customer_date'] - delivered['order_purchase_timestamp']).dt.days
delivered['delivery_delta'] = (delivered['order_delivered_customer_date'] - delivered['order_estimated_delivery_date']).dt.days

print("Delivery Time Analysis (Delivered Orders Only)")
print("="*60)
print(f"Total delivered orders: {len(delivered):,}")
print(f"\nDelivery days statistics:")
print(delivered['delivery_days'].describe())
print(f"\nDelivery delta (actual - estimated):")
print(delivered['delivery_delta'].describe())
print(f"\nEarly deliveries: {(delivered['delivery_delta'] < 0).sum():,} ({(delivered['delivery_delta'] < 0).mean()*100:.1f}%)")
print(f"On-time deliveries: {(delivered['delivery_delta'] == 0).sum():,} ({(delivered['delivery_delta'] == 0).mean()*100:.1f}%)")
print(f"Late deliveries: {(delivered['delivery_delta'] > 0).sum():,} ({(delivered['delivery_delta'] > 0).mean()*100:.1f}%)")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Delivery days histogram
axes[0].hist(delivered['delivery_days'].dropna(), bins=50, edgecolor='white', alpha=0.7)
axes[0].axvline(delivered['delivery_days'].median(), color='red', linestyle='--', label=f'Median: {delivered["delivery_days"].median():.0f} days')
axes[0].set_xlabel('Delivery Days')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Delivery Time Distribution')
axes[0].legend()

# Delivery delta histogram
delta_clipped = delivered['delivery_delta'].clip(-30, 30)  # Clip extremes for visualization
axes[1].hist(delta_clipped.dropna(), bins=60, edgecolor='white', alpha=0.7, color='orange')
axes[1].axvline(0, color='green', linestyle='--', linewidth=2, label='On-time')
axes[1].set_xlabel('Days (Actual - Estimated)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Delivery Delta Distribution (clipped to ±30 days)')
axes[1].legend()

plt.tight_layout()
plt.savefig('outputs/delivery_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

### 5.9 Review Score vs Delivery Performance

In [ ]:
# Merge delivery data with reviews
delivery_reviews = delivered.merge(reviews[['order_id', 'review_score']], on='order_id', how='inner')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot: review score by delivery status
delivery_reviews['delivery_status'] = pd.cut(
    delivery_reviews['delivery_delta'],
    bins=[-np.inf, -1, 0, np.inf],
    labels=['Early', 'On-time', 'Late']
)

sns.boxplot(data=delivery_reviews, x='delivery_status', y='review_score', ax=axes[0], palette='RdYlGn')
axes[0].set_xlabel('Delivery Status')
axes[0].set_ylabel('Review Score')
axes[0].set_title('Review Score by Delivery Status')

# Average review score by delivery delta
delta_review = delivery_reviews.groupby(delivery_reviews['delivery_delta'].clip(-10, 20))['review_score'].mean()
axes[1].plot(delta_review.index, delta_review.values, marker='o', linewidth=2)
axes[1].axvline(0, color='green', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Delivery Delta (days)')
axes[1].set_ylabel('Average Review Score')
axes[1].set_title('Average Review Score by Delivery Delta')

plt.tight_layout()
plt.savefig('outputs/delivery_vs_reviews.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Summary Statistics

In [ ]:
print("\n" + "="*60)
print("DATASET SUMMARY")
print("="*60)

datasets_summary = [
    ('customers', customers, 'Customer master data with unique customer IDs'),
    ('orders', orders, 'Order transactions with timestamps and status'),
    ('order_items', order_items, 'Line-item details (products, prices, freight)'),
    ('payments', payments, 'Payment transactions (type, installments, value)'),
    ('reviews', reviews, 'Customer reviews and ratings'),
    ('products', products, 'Product catalog with categories'),
    ('sellers', sellers, 'Seller information and location'),
    ('geolocation', geolocation, 'Geographic coordinates by ZIP code'),
]

for name, df, desc in datasets_summary:
    print(f"\n{name.upper()}:")
    print(f"  Description: {desc}")
    print(f"  Rows: {len(df):,}")
    print(f"  Columns: {len(df.columns)}")

## 7. Feature Engineering Recommendations

Based on the EDA, here are the recommended features for behavioral clustering:

### RFM Features
- `recency_days`: Days since last purchase
- `frequency`: Number of orders per customer
- `monetary_total`: Total lifetime spend
- `monetary_avg`: Average order value
- `tenure_days`: Days between first and last purchase

### Payment Behavior
- `pct_credit_card`: % of orders paid by credit card
- `pct_boleto`: % of orders paid by boleto
- `avg_installments`: Average number of installments
- `installment_rate`: % of orders with >1 installment

### Category Preferences
- `category_diversity`: Number of distinct categories purchased
- `category_hhi`: Herfindahl concentration index (0=diverse, 1=concentrated)
- `avg_product_price`: Mean unit price of purchases
- `pct_high_price`: % of orders above median price

### Review/Satisfaction
- `avg_review_score`: Mean review score
- `review_variance`: Variance in review scores
- `review_rate`: % of orders with reviews
- `has_comment_rate`: % of reviews with comments

### Delivery Experience
- `avg_delivery_days`: Mean delivery time
- `avg_delivery_delta`: Mean (actual - estimated) delivery
- `pct_late_delivery`: % of orders delivered late

## 8. Scaling Recommendations

For k-means clustering, all features should be standardized (mean=0, std=1) using `StandardScaler`.

**Pre-scaling transformations:**
- **Log transform** for right-skewed monetary features: `monetary_total`, `monetary_avg`, `avg_product_price`
- **No additional transform** for: percentages, scores, counts

**Implementation:**
```python
from sklearn.preprocessing import StandardScaler
import numpy as np

# Log transform skewed features
for col in ['monetary_total', 'monetary_avg', 'avg_product_price']:
    features[f'{col}_log'] = np.log1p(features[col])

# Standardize all features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features[feature_columns])
```

## 9. Data Quality Notes

### Missing Data
- **Reviews**: `review_comment_title` (87% missing), `review_comment_message` (58% missing) - Most customers don't leave text comments
- **Orders**: Delivery timestamps missing for non-delivered orders (~3% of orders)
- **Products**: `product_category_name` missing for 610 products (1.9%)

### Outliers
- **Price**: Range R$0.85 - R$6,735, with heavy right skew. Consider log transform or capping at 99th percentile
- **Freight**: Range R$0 - R$409, also right-skewed
- **Installments**: 1-24, with 52% being single payment

### Key Observations
1. **Low repeat rate**: Only ~3% of customers have >1 order - most behavioral features will be based on single transactions
2. **Strong delivery-satisfaction link**: Late deliveries correlate with lower review scores
3. **Credit card dominance**: 74% of payments are credit card
4. **Positive review bias**: 57% of reviews are 5-star